In [ ]:
import sys
import gensim, logging
import pandas as pd

In [ ]:
import gensim.downloader as api

model = api.load("word2vec-ruscorpora-300")

In [ ]:
def Search_column_content(Content,Search_content,file_name,noun_bonus=1,adj_bonus=1):
    """
    Content - pd.Series с контентом.
    Search_content - List с контентом(!) поиска [горный_NOUN,горный_ADJ,....]  
    
    Выход - по датафрейм (Similarity | id(просто чтобы не просрать) | магазин.csv)
    
    """
    Similarity=[]
    for row in Content:
        #row строка вида велосипед_NOUN,велосипед_ADJ, и тд
        content_words=row.split(',')
        Final_rating=[]

        for word in Search_content:
            coef=1
            if word.split('_')[1]=='NOUN':
                coef=noun_bonus
            else:
                coef=adj_bonus
            ratings=[]
            for cont in content_words:
                #print(word,cont)
                
                ratings.append(model.similarity(word, cont)*coef)
            Final_rating.append(max(ratings))
        Similarity.append(sum(Final_rating))
        
    return pd.DataFrame({'Similarity':Similarity,'ID':range(len(Similarity)),'Shop':file_name})

In [ ]:
Category='Food'
File_names_by_cat={'Sport':['Вставить через запятую'],'Food':['data_with_keyWords.csv','data_with_keyWords1.csv'],'Tablets':[]}
text=['яблоко_NOUN','красное_NOUN','красный_ADJ']

In [ ]:
def Total_search(File_names,Search_content): 
    """
    File_names = File_names_by_cat[Category]
    Search_content - Обработанная строка поиска в виде List
    
    Выход - отсортированный по Similarity descending полный Dataframe вида (Similarity | id(в оригинальном файле) | магазин.csv)
    
    """
    
    All_similarities=[]
    for file in File_names:
        print('NEXT FILE')
        current_file=pd.read_csv(file)
        All_similarities.append(Search_column_content(current_file['words'].iloc[:100],Search_content,file,2,0.75))
    return pd.concat(All_similarities,axis=0).sort_values(by=['Similarity'],ascending =False)

In [ ]:
result=Total_search(File_names_by_cat[Category],text)